In [1]:
! uv pip install pandas numpy seaborn plotly matplotlib dataframe_image pylatex markdown-generator mdutils
! uv run playwright install chromium

Using Python 3.13.3 environment at: /home/tritbool/Workspace/.venv
Audited 9 packages in 24ms
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu24.04-arm64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu24.04-arm64.
BEWARE: your OS is not officially supported by Playwright; downloading fallback build for ubuntu24.04-arm64.


In [78]:
import pandas as pd
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import dataframe_image as dfi
import pylatex as pl
from pylatex import Command, Document, Section, Subsection
from pylatex.utils import NoEscape, italic
from mdutils.mdutils import *
from mdutils import *
import os

In [79]:
trip_types={
    0:'commute',
    1:'offroad',
    2:'trail',
    3:'greestyle',
    4:'course',
    }

trip_colors={
    trip_types.get(0):"background-color: orange",
    trip_types.get(1):"background-color: olivedrab",
    trip_types.get(2):"background-color: gainsboro",
    trip_types.get(3):"background-color: lightskyblue",
    trip_types.get(4):"background-color: yellow",
    'euc_world':'background-color: mediumpurple',
    'wheel_log':"background-color: pink",
}

In [80]:
def single_chart(data,x,y)-> str:
    try:
        fig = px.line(data, x=x, y=y)
        return fig.to_html(full_html=False, include_plotlyjs='cdn')
    except:
        return "<div> NO "+y.upper()+" DATA </div>"

In [81]:
def dual_chart(data,title,x,y1,y2)->str:
    try:
        fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    
        fig.add_trace(
            go.Scatter(x=data[x], y=data[y1], name=y1),
            row=1, col=1, secondary_y=False
        )
    
        fig.add_trace(
            go.Scatter(x=data[x], y=data[y2], name=y2),
            row=1, col=1, secondary_y=True
        )
        fig.update_layout(title=title)
        return fig.to_html(full_html=False, include_plotlyjs='cdn')
    except:
        return "<div> NO "+title.upper()+" DATA </div>"

# CONFIGURATION

In [82]:
trips_root="trips/"
brands = os.listdir(trips_root)
brands = [ brand for brand in brands if not "ipynb" in brand]

for brand in brands:
    wheels = os.listdir(trips_root+brand)
    wheels = [ wheel for wheel in wheels if not "ipynb" in wheel]
    for wheel in wheels:
        trips = os.listdir(trips_root+brand+"/"+wheel)
        trips = [ trip for trip in trips if not "ipynb" in trip]
        for trip in trips:
            meta = pd.read_csv(trips_root+brand+"/"+wheel+"/"+trip+"/meta.csv")
            wheel=meta["wheel"][0]
            brand=meta["brand"][0]
            year=meta["year"][0]
            app=meta["app"][0]
            trip_type=trip_types.get(int(meta["trip_type"][0]))
            rider_weight=float(meta["rider_weight"][0])
            temperature=float(meta["temperature"][0])
            tire_pressure_bar=float(meta["tire_pressure"][0])
            wind=meta["wind"][0]
            rain=bool(meta["rain"][0])
            csv=trips_root+brand+"/"+wheel+"/"+trip+"/"+trip+".csv"
            analyze_run(wheel,csv)

0
0
1
2
3
0


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



0


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



1


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



0


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



0
1


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/numpy/_core/_methods.py:51: RuntimeWarning:

invalid value encountered in reduce



'z10'

In [6]:
wheel="blitz"
brand="Begode"
year="2024"
app='euc_world'
csv='blitz'


trip_type=trip_types.get(0)
rider_weight=85
temperature=20
tire_pressure_bar=2.5
wind='low'
rain=False


In [77]:
def analyze_run(wheel,csv):
    stats_dir="stats/"+wheel
    analysis_dir='analysis/'+wheel
    
    trip_id=0
    if not os.path.exists(stats_dir):
        os.mkdir(stats_dir)
    else:
        trip_ids = pd.read_csv(stats_dir+"/env.csv")
        trip_ids=trip_ids["trip_id"]
        trip_id=trip_ids.values.max()+1
        
    analysis_file=analysis_dir+'/'+wheel+"_"+str(trip_id)
    
    if not os.path.exists(analysis_dir):
        os.mkdir(analysis_dir)
    
    print(trip_id)
    
    wheel_specs = pd.read_csv("specs/"+wheel+".csv",sep='|')
    battery_cap=wheel_specs[wheel_specs["Field"]=="Battery"]
    battery_cap=battery_cap.values
    battery_cap=battery_cap[0][1].split("Wh")[0]
    battery_cap=int(battery_cap)/1000

    wheel_weight=wheel_specs[wheel_specs["Field"]=="Net weight"]
    wheel_weight=wheel_weight.values
    wheel_weight=float(wheel_weight[0][1].split("kg")[0])
    
    data=pd.read_csv(csv,low_memory=False)
    data["wh-km"]=data["power"]/data["speed"]
    
    filtered=data[(data.speed>3.0) & (data.speed<65.0)]
    
    battery_ratio=data["battery"].values[5] if app == 'euc_world' else data["battery_level"].values[5] if app == "wheel_log" else 1
    battery_status=battery_cap*battery_ratio/100
    alt_diff=round(data["gps_alt"].max()-data["gps_alt"].min(),1) if app == "euc_world" else None
    dist=[data["distance"].values[-1]] if app == "euc_world" else  [data["distance"].values[-1]/1000] if app == "wheel_log" else 0

    env_info={
        'data_origin':app,
        'trip_id':trip_id,
        'trip_type':trip_type,
        'trip_distance_km': dist,
        'rider_weight_kg':[rider_weight],
        'wheel_weight_kg':[wheel_weight],
        'global_weight_kg':[rider_weight+wheel_weight],
        'tire_pressure_bars':[tire_pressure_bar],
        'outdoor_temperature_c': [temperature],
        'wind': [wind],
        'rain': [rain],
        'altitude_difference':[alt_diff]
    }
    df_env = pd.DataFrame(env_info)
    df_env.to_csv(stats_dir+"/env.csv", mode='a', index=False,header=(trip_id==0))
    df_env_styled = df_env.style.apply(lambda x: [trip_colors.get(val) for val in x], subset=['trip_type','data_origin'])

    susbet=None
    gp=None
    if app == "wheel_log":
        data['time_corrected']=list(map(lambda x: x.split('.')[0],data['time'].values))
        subset=data[['speed', 'voltage', 'phase_current', 'current', 'power',
           'torque', 'pwm', 'battery_level', 'distance', 'totaldistance',
           'system_temp', 'time_corrected','wh-km']]
        gp=subset.groupby("time_corrected").mean()
        
    elif app == "euc_world":
        subset=data[['gps_datetime','duration', 'duration_riding', 'distance', 'distance_total','speed', 'speed_avg', 'speed_avg_riding',
                 'speed_max','acceleration', 'voltage', 'current', 'current_phase', 'power','battery', 'temp','safety_margin','wh-km']]
        gp=subset.groupby("gps_datetime").mean()
        
    gp["kwh"]=gp["power"]/3600000

    batt_stat=[battery_status]
    batt_lvl=[data["battery"].values[-1] if app=="euc_world" else data["battery_level"].values[-1]]
    batt_v=[data["voltage"].values[-1]]
    batt_lvl_est=[round((battery_status-round(gp[["kwh"]].sum().kwh,2))/battery_cap,2)*100]
    conso=[round(gp[gp["kwh"]>0].sum().kwh,2)]
    regen=[round(gp[gp["kwh"]<0].sum().kwh,2)]
    conso_cor=[round(gp[["kwh"]].sum().kwh,2)]
    avg_whkm=[round(filtered["wh-km"].mean(),2)]
    a = np.array(data["power"])
    a=np.nan_to_num(a)
    b = np.array(data["voltage"])
    b=np.nan_to_num(b)
    cross_corr = np.corrcoef(a, b)
    cc=round(cross_corr[0][1],3)

    stats_conso = {
        'trip_id':trip_id,
        'initial_battery_kwh': batt_stat,
        'end_batt_lvl_pct': batt_lvl,
        'end_batt_voltage': batt_v,
        'estimated_end_batt_lvl_pct':batt_lvl_est,
        'conso_kwh':conso,
        'regen_kwh':regen,
        'conso_corrected_kwh':conso_cor,
        'average_wh/km':avg_whkm,
        'voltage_sag_coeff':cc
    }
    df_conso = pd.DataFrame(stats_conso)
    df_conso.to_csv(stats_dir+"/conso.csv", mode='a', index=False, header=(trip_id==0))
    df_conso_styled = df_conso.style.apply(lambda x: ['background-color: lightcoral' if val > 25.0 or val < -0.5 else 'background-color: orange' if  val > 15 else 'background-color: lightgreen' for val in x], subset=['average_wh/km','voltage_sag_coeff'])

    wh=None
    speed=None
    pwr=None
    cur=None
    cur_p=None
    temp=None
    motor_temp=None
    batt_temp=None
    saf=None
    
    if app == "euc_world":
        wh=[filtered["wh-km"].min(),filtered["wh-km"].max(),filtered["wh-km"].mean(),filtered["wh-km"].std()]
        speed=[filtered["speed"].min(),filtered["speed"].max(),filtered["speed"].mean(),filtered["speed"].std()]
        pwr=[filtered["power"].min(),filtered["power"].max(),filtered["power"].mean(),filtered["power"].std()]
        cur=[filtered["current"].min(),filtered["current"].max(),filtered["current"].mean(),filtered["current"].std()]
        cur_p=[filtered["current_phase"].min(),filtered["current_phase"].max(),filtered["current_phase"].mean(),filtered["current_phase"].std()]
        temp=[filtered["temp"].min(),filtered["temp"].max(),filtered["temp"].mean(),filtered["temp"].std()]
        motor_temp=[filtered["temp_motor"].min(),filtered["temp_motor"].max(),filtered["temp_motor"].mean(),filtered["temp_motor"].std()]
        batt_temp=[filtered["temp_batt"].min(),filtered["temp_batt"].max(),filtered["temp_batt"].mean(),filtered["temp_batt"].std()]
        saf=[filtered["safety_margin"].min(),filtered["safety_margin"].max(),filtered["safety_margin"].mean(),filtered["safety_margin"].std()]
    elif app == "wheel_log":
        wh=[filtered["wh-km"].min(),filtered["wh-km"].max(),filtered["wh-km"].mean(),filtered["wh-km"].std()]
        speed=[filtered["speed"].min(),filtered["speed"].max(),filtered["speed"].mean(),filtered["speed"].std()]
        pwr=[filtered["power"].min(),filtered["power"].max(),filtered["power"].mean(),filtered["power"].std()]
        cur=[filtered["current"].min(),filtered["current"].max(),filtered["current"].mean(),filtered["current"].std()]
        cur_p=[filtered["phase_current"].min(),filtered["phase_current"].max(),filtered["phase_current"].mean(),filtered["phase_current"].std()]
        temp=[filtered["system_temp"].min(),filtered["system_temp"].max(),filtered["system_temp"].mean(),filtered["system_temp"].std()]

    summary = {
        'trip_id':[trip_id,trip_id,trip_id,trip_id],
        'wh.km-1': wh,
        'speed': speed,
        'power': pwr,
        'current': cur,
        'current_phase':cur_p,
        'temperature':temp,
        'motor_temperature':motor_temp,
        'battery_temperature':batt_temp,
        'safety_margin':saf,
    }
    df = pd.DataFrame(summary, index=['min', 'max', 'mean', 'std_dev'])
    df.index.name = 'stat'
    df.to_csv(stats_dir+"/summary.csv", mode='a', header=(trip_id==0))
    df_styled=df.style.background_gradient()

    dt='gps_datetime' if app == "euc_world" else  'time_corrected' if app == "wheel_log" else ""
    charts={
        "power":single_chart(data,dt,'power'),
        "current":single_chart(data,dt,'current'),
        "current_phase":single_chart(data,dt,'current_phase' if app == "euc_world" else  'phase_current' if app == "wheel_log" else ""),
        "temp_motor":single_chart(data,dt,'temp_motor'),
        "temp_batt":single_chart(data,dt,'temp_batt'),
        "temp":single_chart(data,dt,'temp' if app == "euc_world" else  'system_temp' if app == "wheel_log" else ""),
        "voltage":single_chart(data,dt,'voltage'),
        "battery":single_chart(data,dt,'battery' if app == "euc_world" else  'battery_level' if app == "wheel_log" else ""),
        "alarm":single_chart(data,dt,'alarm'),
        "acceleration":single_chart(data,dt,'acceleration'),
        "power":single_chart(data,dt,'power'),
        "voltage_sag":dual_chart(data,'voltage sag',dt,'power','voltage'),
        "safety_margin":dual_chart(data,'safety margin',dt,'power','safety_margin'),
    }
    mdFile = MdUtils(file_name=analysis_file,title='')
    # Additional Markdown syntax...
    
    mdFile.new_header(level=1, title="Trip details")
    mdFile.write(df_env_styled.to_html())
    mdFile.new_line()
    
    mdFile.new_header(level=1, title="Summary")
    mdFile.write(df_styled.to_html())
    mdFile.new_line()
    
    mdFile.new_header(level=1, title="Consumption")
    mdFile.write(df_conso_styled.to_html())
    mdFile.new_line()
    
    mdFile.new_header(level=1, title="Charts")
    
    
    for chart in charts:
        mdFile.new_line()
        mdFile.new_header(level=2, title=chart)
        mdFile.write(charts.get(chart))
    
    mdFile.create_md_file()